In [1]:
import os
import numpy as np
import glob
import csv
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats
import patsy
from scipy.optimize import minimize
from scipy.optimize import basinhopping
from sklearn import linear_model
import multiprocessing
import random
from scipy.stats import norm
from scipy.stats import beta
%matplotlib inline

In [57]:
#mess around with this to get a sense of the dataframe
home_dir = '/Users/ianballard/Dropbox/two_step_analysis/'
data_files = glob.glob(home_dir + '*csv') #all files ending in csv

for f in data_files:
    data = pd.read_csv(f)
data = data.drop(['view_history','stimulus','internal_chunk_id'],1) #drop useless columns

print data.tail(3) #last three rows (see head for first)

        rt  key_press   trial_type  trial_index  trial_index_global  \
1295    -1         -1  single-stim            0                1295   
1296    -1         -1  single-stim            0                1296   
1297  2041         13         text            0                1297   

      time_elapsed              exp_id condition  trial_num       trial_id  \
1295       1042510  two-stage-decision         1        199  test_FB_stage   
1296       1043512                 NaN       NaN        NaN            NaN   
1297       1045556                 NaN       NaN        NaN            NaN   

                                               FB_probs  
1295  0.4550775204592807,0.3932520630940464,0.360271...  
1296                                                NaN  
1297                                                NaN  


In [54]:
ntrials = 200
key_map = {37:0,39:1} #keystrokes. Remember that the location of the stimuli changes trial to trial
state_map = {'3,2':1,'5,4':2,'2,3':1,'4,5':2} #
practice = ['practice_first_stage', 'practice_second_stage', 'practice_FB_stage'] #labels for practice trials

for f in data_files:

    sub_id = f.split('/')[-1].split('.csv')[0] #parse subid from filename
    
    #dictionary to fill
    data = {'rt1':[],'rt2':[],'action1':[],'action2':[],'choice1':[],
            'choice2':[],'state':[],'rew':[],'ttype':[],'stay':[],'sub':[sub_id] * ntrials} 
    
    #preprocess data
    df = pd.read_csv(f) 
    df = df.drop(['view_history','stimulus','internal_chunk_id'],1) #drop useless columns

    #drop practice trials
    for t in practice: 
        trial_df = trial_df[trial_df['trial_id'] != t]
            
    #get choices, actions, rewards, RTs, etc from each trial
    for i in range(ntrials):

        trial_df = df[df['trial_num'] == i] #get data corresponding to that trial
        
        state = state_map[str(trial_df['condition'].values[1])]
        data['rt1'].append(trial_df['rt'].values[0])
        data['rt2'].append(trial_df['rt'].values[1])
        data['rew'].append(trial_df['condition'].values[2]) 
        data['state'].append(state)

        action1 = key_map[trial_df['key_press'].values[0]]
        action2 = key_map[trial_df['key_press'].values[1]]
        choice1 = int(trial_df['condition'].values[0].split(',')[action1])
        choice2 = int(trial_df['condition'].values[1].split(',')[action2])
        
        data['action1'].append(action1)
        data['action2'].append(action2)
        data['choice1'].append(choice1)
        data['choice2'].append(choice2)
        
        #determine if it was a switch or stay trial
        if len(data['choice1'])>0 and (choice1 == data['choice1'][-1]):
                data['stay'].append('stay')
        else:
            data['stay'].append('switch')
        
        #determine if it was a common or rare transition
        if (choice1 == 0 and state == 1) or (choice1 == 1 and state == 2):
            data['ttype'].append('common')
        else:
            data['ttype'].append('rare')
        
data = pd.DataFrame(data) #make dataFrame

In [55]:
print data.head(10)

   action1  action2  choice1  choice2 rew   rt1   rt2  state  stay  \
0        0        0        1        5   0   843  1628      2  stay   
1        1        0        1        2   0  1102   793      1  stay   
2        1        0        1        3   0  1456   479      1  stay   
3        1        1        1        3   0  1436   689      1  stay   
4        0        0        1        5   1   567   847      2  stay   
5        0        0        1        5   0   982   352      2  stay   
6        0        0        0        2   1  1110   351      1  stay   
7        0        0        1        4   1   424   399      2  stay   
8        0        0        1        3   1   446   310      1  stay   
9        0        0        0        2   1   462   337      1  stay   

           sub   ttype  
0  a.moretti18  common  
1  a.moretti18    rare  
2  a.moretti18    rare  
3  a.moretti18    rare  
4  a.moretti18  common  
5  a.moretti18  common  
6  a.moretti18  common  
7  a.moretti18  common  
8  a

In [5]:
# #Ignore this for now. This is for paying the real mechanical turk workers

# ##takes the mturk results file and makes a dictionary mapping
# ##the worker id to the assignment id
# def get_assignment_id(assignment_file):
# 	rowNum=0
# 	worker_id_2_assignment_id = {}
# 	with(open(assignment_file,'rU')) as f:
# 		reader = csv.reader(f)
# 		#worker id is 15 and 14
# 		for row in reader:
# 	 		if rowNum > 0:
# 				worker_id_2_assignment_id[row[15]] = row[14]
# 			rowNum += 1
# 	return worker_id_2_assignment_id

In [34]:
# ##payment  
# assignment_file = os.path.join(home_dir,'results/Batch_2088213_batch_results.csv')
# payment_file = os.path.join(home_dir,'results/payment1.txt')
# worker_id_2_assignment_id = get_assignment_id(assignment_file)
# bonus = 4.00

# data_f = glob.glob(home_dir + '*csv')
# for f in data_f:
#     worker = f.split('/')[-1][:-4]
#     if worker in worker_id_2_assignment_id.keys():
#         out_str = [worker_id_2_assignment_id[worker] ,worker , str(bonus)]
#         pay = open(payment_file,'a')
#         pay.write('\t'.join(out_str))
#         pay.write('\n')
